## Part 2. Model Training & Evaluation - RNN

Now with the pretrained word embeddings acquired from Part 1 and the dataset acquired from Part
0, you need to train a deep learning model for topic classification using the training set, conforming
to these requirements:
- Use the pretrained word embeddings from Part 1 as inputs, together with your implementation
in mitigating the influence of OOV words; make them learnable parameters during training
(they are updated).
- Design a simple recurrent neural network (RNN), taking the input word embeddings, and
predicting a topic label for each sentence. To do that, you need to consider how to aggregate
the word representations to represent a sentence.
- Use the validation set to gauge the performance of the model for each epoch during training.
You are required to use accuracy as the performance metric during validation and evaluation.
- Use the mini-batch strategy during training. You may choose any preferred optimizer (e.g.,
SGD, Adagrad, Adam, RMSprop). Be careful when you choose your initial learning rate and
mini-batch size. (You should use the validation set to determine the optimal configuration.)
Train the model until the accuracy score on the validation set is not increasing for a few
epochs.
- Try different regularization techniques to mitigate overfitting.
- Evaluate your trained model on the test dataset, observing the accuracy score.

In [1]:
import json
import numpy as np
import random
import itertools
from pathlib import Path
from torchtext import data, datasets
from torch.utils.data import TensorDataset, DataLoader
from utils.config import Config
from utils.train import train_rnn_model_with_parameters
from utils.helper import SentenceDataset, collate_fn

/home/linnsheng/Desktop/NTU/S3/Y1/NLP/SC4002/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_sm', include_lengths=True)
LABEL = data.LabelField()

train_data, test_data = datasets.TREC.splits(TEXT, LABEL, fine_grained=False)

In [3]:
train_data, valid_data = train_data.split(random_state=random.seed(Config.SEED), split_ratio=0.8)

In [4]:
TEXT.build_vocab(train_data, vectors="glove.6B.300d")
LABEL.build_vocab(train_data)

### Import the embedding matrix and vocab index mapping (train data)

In [5]:
embedding_path = Path("models/embedding_matrix.npy")
index_from_word_path = Path("models/index_from_word.json")

embedding_matrix = np.load(embedding_path)
with index_from_word_path.open() as f:
    index_from_word = json.load(f)

In [6]:
train_dataset = SentenceDataset(train_data.examples, index_from_word, LABEL.vocab)
valid_dataset = SentenceDataset(valid_data.examples, index_from_word, LABEL.vocab)
test_dataset = SentenceDataset(test_data.examples, index_from_word, LABEL.vocab)        

### Dataset

In [7]:
SEARCH_SPACE = {
    "batch_size": [32, 64, 128, 256, 512, 1024, 2048],
    "learning_rate": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    "optimizer_name": ["SGD", "Adagrad", "RMSprop", "Adam"],
    "hidden_dim": [256, 128, 64, 32],
    "num_layers": [1, 2, 4],
    "sentence_representation_type": ["last", "average", "max"],
}
all_combinations = list(itertools.product(
    SEARCH_SPACE["batch_size"],
    SEARCH_SPACE["learning_rate"],
    SEARCH_SPACE["optimizer_name"],
    SEARCH_SPACE["hidden_dim"],
    SEARCH_SPACE["num_layers"],
    SEARCH_SPACE["sentence_representation_type"]
))

In [8]:
# train_rnn_model_with_parameters(
# embedding_matrix=embedding_matrix,
#     train_dataset=train_dataset,
#     val_dataset=valid_dataset,
#     batch_size=128,
#     learning_rate=5e-4,
#     optimizer_name='Adam',
#     hidden_dim=128,
#     num_layers=1,
#     sentence_representation_type='average',
#     freeze_embedding=False,
#     show_progress=True,
# )

In [9]:
# for batch_size, lr, optimizer_name, hidden_dim, num_layers, sr_type in all_combinations:
#     print(f"Training with configuration: batch_size={batch_size}, lr={lr}, optimizer={optimizer_name}, "
#           f"hidden_dim={hidden_dim}, num_layers={num_layers}, sentence_repr={sr_type}")

#     train_rnn_model_with_parameters(
#         embedding_matrix=embedding_matrix,
#         train_dataset=train_dataset,
#         val_dataset=valid_dataset,
#         batch_size=batch_size,
#         learning_rate=lr,
#         optimizer_name=optimizer_name,
#         hidden_dim=hidden_dim,
#         num_layers=num_layers,
#         sentence_representation_type=sr_type,
#         freeze_embedding=False,
#         show_progress=True,
#     )

In [ ]:
for hidden_dim in SEARCH_SPACE["hidden_dim"]:
    for num_layers in SEARCH_SPACE["num_layers"]:
        for optimizer_name in SEARCH_SPACE["optimizer_name"]:
            for batch_size in SEARCH_SPACE["batch_size"]:
                for learning_rate in SEARCH_SPACE["learning_rate"]:
                    for sentence_representation_type in SEARCH_SPACE["sentence_representation_type"]:
                        log_message = f"---------- batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers}; sentence_representation_{sentence_representation_type}  ----------"
                        print(log_message)
                        train_rnn_model_with_parameters(
                            embedding_matrix=embedding_matrix,
                            train_dataset=train_dataset,
                            val_dataset=valid_dataset,
                            batch_size=batch_size,
                            learning_rate=learning_rate,
                            optimizer_name=optimizer_name,
                            hidden_dim=hidden_dim,
                            num_layers=num_layers,
                            sentence_representation_type=sentence_representation_type,
                            show_progress=True,
                            freeze_embedding=False,
                        )

Seed set to 42
/home/linnsheng/Desktop/NTU/S3/Y1/NLP/SC4002/.venv/lib/python3.13/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'rnn_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['rnn_model'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


---------- batch_size_32; lr_1e-05; optimizer_SGD; hidden_dim_256; num_layers_1; sentence_representation_last  ----------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 2.6 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.418    Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Epoch 0:   0%|          | 0/137 [00:00<?, ?it/s]                           Logits range: [-0.140, 0.144]
Loss: 1.793
Predictions: tensor([4, 4, 4, 4, 3], device='cuda:0')
True labels: tensor([0, 1, 1, 1, 0], device='cuda:0')
Epoch 1:   0%|          | 0/137 [00:00<?, ?it/s, v_num=0, train_loss=1.820, train_acc=0.0667, val_loss=1.810, val_acc=0.151]          Logits range: [-0.117, 0.130]
Loss: 1.786
Predictions: tensor([4, 3, 4, 4, 4], device='cuda:0')
True labels: tensor([0, 1, 1, 1, 2], device='cuda:0')
Epoch 2:   0%|          | 0/137 [00:00<?, ?it/s, v_num=0, train_loss=1.770, train_acc=0.250, val_loss=1.810, val_acc=0.150]            Logits range: [-0.148, 0.162]
Loss: 1.803
Predictions: tensor([4, 3, 4, 4, 4], device='cuda:0')
True labels: tensor([0, 5, 0, 3, 4], device='cuda:0')
Epoch 3:   0%|          | 0/137 [00:00<?, ?it/s, v_num=0, train_loss=1.730, train_acc=0.375, val_loss=1.810, val_acc=0.151]            Logits range: [-0.117, 0.127]
Loss: 1.785
Predictions: tensor([3, 4, 4,

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 2.6 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.418    Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


...

---------- batch_size_64; lr_1e-05; optimizer_RMSprop; hidden_dim_256; num_layers_1; sentence_representation_last  ----------
Epoch 0:   0%|          | 0/69 [00:00<?, ?it/s]                             Logits range: [-0.173, 0.149]
Loss: 1.794
Predictions: tensor([4, 4, 4, 4, 4], device='cuda:0')
True labels: tensor([0, 0, 1, 0, 1], device='cuda:0')
Epoch 1:   0%|          | 0/69 [00:00<?, ?it/s, v_num=0, train_loss=1.740, train_acc=0.0833, val_loss=1.740, val_acc=0.197]         Logits range: [-0.423, 0.167]
Loss: 1.737
Predictions: tensor([1, 3, 1, 1, 1], device='cuda:0')
True labels: tensor([0, 0, 1, 0, 4], device='cuda:0')
Epoch 2:   0%|          | 0/69 [00:00<?, ?it/s, v_num=0, train_loss=1.710, train_acc=0.200, val_loss=1.710, val_acc=0.188]         Logits range: [-0.656, 0.225]
Loss: 1.710
Predictions: tensor([1, 0, 1, 0, 1], device='cuda:0')
True labels: tensor([1, 1, 1, 0, 5], device='cuda:0')
Epoch 3:   0%|          | 0/69 [00:00<?, ?it/s, v_num=0, train_loss=1.710, train_acc=

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 2.6 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.418    Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


---------- batch_size_64; lr_1e-05; optimizer_RMSprop; hidden_dim_256; num_layers_1; sentence_representation_average  ----------
Epoch 0:   0%|          | 0/69 [00:00<?, ?it/s]                             Logits range: [-0.088, 0.123]
Loss: 1.801
Predictions: tensor([4, 4, 4, 4, 4], device='cuda:0')
True labels: tensor([0, 0, 1, 0, 1], device='cuda:0')
Epoch 1:   0%|          | 0/69 [00:00<?, ?it/s, v_num=0, train_loss=1.780, train_acc=0.133, val_loss=1.780, val_acc=0.225]         Logits range: [-0.234, 0.110]
Loss: 1.773
Predictions: tensor([1, 1, 3, 4, 3], device='cuda:0')
True labels: tensor([0, 0, 1, 0, 4], device='cuda:0')
Epoch 2:   0%|          | 0/69 [00:00<?, ?it/s, v_num=0, train_loss=1.740, train_acc=0.550, val_loss=1.760, val_acc=0.243]         Logits range: [-0.482, 0.204]
Loss: 1.763
Predictions: tensor([1, 1, 1, 1, 1], device='cuda:0')
True labels: tensor([1, 1, 1, 0, 5], device='cuda:0')
Epoch 3:   0%|          | 0/69 [00:00<?, ?it/s, v_num=0, train_loss=1.710, train_ac

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 2.6 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.418    Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


---------- batch_size_64; lr_1e-05; optimizer_RMSprop; hidden_dim_256; num_layers_1; sentence_representation_max  ----------
Epoch 0:   0%|          | 0/69 [00:00<?, ?it/s]                             Logits range: [-0.215, 0.182]
Loss: 1.806
Predictions: tensor([3, 3, 3, 3, 3], device='cuda:0')
True labels: tensor([0, 0, 1, 0, 1], device='cuda:0')
Epoch 1:   0%|          | 0/69 [00:00<?, ?it/s, v_num=0, train_loss=1.760, train_acc=0.208, val_loss=1.740, val_acc=0.193]         Logits range: [-0.347, 0.159]
Loss: 1.735
Predictions: tensor([0, 0, 1, 1, 1], device='cuda:0')
True labels: tensor([0, 0, 1, 0, 4], device='cuda:0')
Epoch 2:   0%|          | 0/69 [00:00<?, ?it/s, v_num=0, train_loss=1.700, train_acc=0.400, val_loss=1.710, val_acc=0.197]         Logits range: [-0.574, 0.227]
Loss: 1.715
Predictions: tensor([0, 1, 1, 0, 1], device='cuda:0')
True labels: tensor([1, 1, 1, 0, 5], device='cuda:0')
Epoch 3:   0%|          | 0/69 [00:00<?, ?it/s, v_num=0, train_loss=1.710, train_acc=0.